In [1]:
# !pip install pycaret
# !pip install torch 
# !pip install torchvision
# !pip install transformers 
# !pip install xgboost
# !pip install catboost

# ! pip install \
#     --extra-index-url=https://pypi.nvidia.com \
#     cudf-cu12==24.8.* dask-cudf-cu12==24.8.* cuml-cu12==24.8.* \
#     cugraph-cu12==24.8.* cuspatial-cu12==24.8.* cuproj-cu12==24.8.* \
#     cuxfilter-cu12==24.8.* cucim-cu12==24.8.* pylibraft-cu12==24.8.* \
#     raft-dask-cu12==24.8.* cuvs-cu12==24.8.*

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
%load_ext cudf.pandas
%reload_ext cudf.pandas

In [3]:
import pandas as pd 
pd.set_option('display.max_columns', None)  
import matplotlib.pyplot as plt 
from PIL import Image, ImageFile, UnidentifiedImageError
import os
import pycaret 
ImageFile.LOAD_TRUNCATED_IMAGES = True 
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision 
from torchvision import datasets, models, transforms 
from tempfile import TemporaryDirectory 
from torch.utils.data import DataLoader 
from torch.utils.data import Dataset
from torchvision.models import resnet18, resnet50, VGG19_Weights
import warnings
warnings.filterwarnings("ignore")  
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# from sklearn.model_selection import train_test_split
# import mlflow                                   
# import mlflow.pytorch
# Ignorer tous les avertissements
# from torchmetrics.classification import Accuracy
# from efficientnet_pytorch import EfficientNet
# import numpy as np 
# import seaborn as sns
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# import pretrainedmodels 

In [4]:
# Pour voir l'utilisation de la memoire 
import psutil
import os

def get_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    return memory_info.rss  # Mémoire utilisée en bytes 

In [5]:
#'/root/Gohou/Data/Tabes/New_trans/df_global.csv'
data = pd.read_csv('/root/Gohou/Data/Tabes/New_trans/df_global.csv')
data = data.drop(['Unnamed: 0','Label', 'CODEBRAN'], axis =1)
df = data.copy()

# Gohou/Data/Tabes/New_trans/df_global .csv

In [6]:
df.shape

(6618, 19)

In [7]:
%%cudf.pandas.profile
""" Cette fonction nous permet d'avoir une liste des tous les chemins des images et une autres qui contient l'adressedes images dselon qu'elles soient stockées dans la base de donnees des images mais aussi dans la table fusionnée """
dossier = "/root/Gohou/Data/images/photo_extracted/Photos_sinistres/photos"
import os 
def list_image_files(directory):
    files = [] #le chemins complet    
    adresse_images = [] #l'images elle meme     
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):# Modifier selon les extensions d'images      
            files.append(os.path.join(directory, filename))
            adresse_images.append(filename)
    return files, adresse_images 
chemins, images = list_image_files(dossier) 
print(len(chemins), len(images)) 

130933 130933


                                                                                              
                                  Total time elapsed: 2.691 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [8]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Création des boîtes à moustaches pour chaque groupe
# sns.boxplot(x='cluster', y= df['COUTS_SINISTRES'], data= df)

# # Ajout de labels et titre
# plt.title('Boîtes à Moustaches des Scores par Groupe')
# plt.xlabel('Groupe')
# plt.ylabel('Score')

# # Affichage du graphique 
# plt.show()

In [9]:
def split_dataframe_data(dataset, batch_size):
    dfs = []
    for i in range(0, len(dataset), batch_size):
        df = dataset.iloc[i:i + batch_size]
        dfs.append(df) 
    return dfs
dff = split_dataframe_data(df, 200)  

In [10]:
import numpy as np 

In [11]:
len(dff)

34

In [ ]:
%%cudf.pandas.profile
print(f"Memory usage before: {get_memory_usage() / (1024 * 1024):.2f} MB")

import gc
import os
from PIL import Image

def split_dataframe_into_batches(dataset, batch_size):
    for i in range(0, len(dataset), batch_size):
        yield dataset.iloc[i:i + batch_size]

def process_image(x):  
    full_path = os.path.join(dossier, x) if isinstance(x, str) else x
    if full_path in chemins and full_path not in treated_paths:
        treated_paths.add(full_path) 
        try:
            if os.path.exists(full_path):
                with Image.open(full_path) as img:  # Use 'with' to ensure the image is closed after processing
                    return img.copy()  # Return a copy to keep the image after the 'with' block
            else:
                print(f"Image not found: {full_path}")
                return np.nan
        except Exception as e:
            print(f"Error loading image {full_path}: {e}")
            return np.nan
    else:
        return np.nan 
treated_paths = set() 

batch_df2 = pd.DataFrame() 
for batch_df in split_dataframe_into_batches(dff[21], 10):  # Adjust batch size if necessary
    for i in range(1,5):
        batch_df.loc[:,f'file_uploaded_photo_{i}'] = batch_df.loc[:,f'file_uploaded_photo_{i}'].apply(process_image)
    batch_df2 = pd.concat([batch_df2, batch_df], ignore_index=True)
    del batch_df 
    gc.collect()  

print(f"Memory usage after: {get_memory_usage() / (1024 * 1024):.2f} MB")  

Memory usage before: 1026.14 MB


In [ ]:
batch_df2.dropna(inplace=True) 

In [ ]:
%%cudf.pandas.profile
print(f"Memory usage before: {get_memory_usage() / (1024 * 1024):.2f} MB")
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn

resnet_model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
resnet_model.fc = nn.Identity()  # Remove the final classification layer

image_transforms = transforms.Compose([ 
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])  

def image_to_features(col): 
    image = image_transforms(col).unsqueeze(0)
    features = resnet_model(image) 
    return features.squeeze(0) 

batch_df3 = pd.DataFrame() 
for batch_df in split_dataframe_into_batches(batch_df2, 10):  # Adjust batch size if necessary
    for i in range(1,5):
        batch_df.loc[:, f'feature_images_{i}'] = batch_df.loc[:,f'file_uploaded_photo_{i}'].apply(image_to_features) 
    batch_df3 = pd.concat([batch_df3, batch_df], ignore_index=True)
    del batch_df 
    gc.collect()  
print(f"Memory usage after: {get_memory_usage() / (1024 * 1024):.2f} MB") 

### Bon avec CPU

In [ ]:
# %%cudf.pandas.profile
# # import rmm
# # rmm.reinitialize(pool_allocator=True, managed_memory=False)
# print(f"Memory usage before: {get_memory_usage() / (1024 * 1024):.2f} MB")
# import logging
# from transformers import logging as transformers_logging
# from transformers import BertTokenizer, BertModel
# import torch 

# # Configurer le niveau de logging pour ignorer les avertissements
# transformers_logging.set_verbosity_error() 
 
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased').to(device) 
# # bert_model = BertModel.from_pretrained("bert-base-uncased", torch_dtype=torch.float16, attn_implementation="sdpa")

# def text_to_features(text): 
#     inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
#     outputs = bert_model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).squeeze(0)  # squeeze(0) pour revoir la dimension

# batch_df4 = pd.DataFrame() 
# for batch_df in split_dataframe_into_batches(batch_df3, 10):  # Adjust batch size if necessary
#     batch_df.loc[:,'material_damage'] = batch_df.loc[:,'material_damage'].apply(text_to_features)
#     batch_df.loc[:,'circonstance_detail'] = batch_df.loc[:,'circonstance_detail'].apply(text_to_features)
#     batch_df.loc[:,'disaster_place'] = batch_df.loc[:,'disaster_place'].apply(text_to_features)
#     batch_df4 = pd.concat([batch_df4, batch_df], ignore_index=True)
#     del batch_df 
#     gc.collect()  
# print(f"Memory usage after: {get_memory_usage() / (1024 * 1024):.2f} MB") 

### BON sur GPU mais avec le modele de BERT ( Embbeding de dimension 768) plus de caracteristique

In [ ]:
# %%cudf.pandas.profile
# # import rmm
# # rmm.reinitialize(pool_allocator=True, managed_memory=False)

# print(f"Memory usage before: {get_memory_usage() / (1024 * 1024):.2f} MB")
# import logging
# from transformers import logging as transformers_logging
# from transformers import BertTokenizer, BertModel
# import torch
# import pandas as pd
# import gc

# # Configurer le niveau de logging pour ignorer les avertissements
# transformers_logging.set_verbosity_error()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

# def text_to_features(text):
#     inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
#     inputs = {key: val.to(device) for key, val in inputs.items()}  # Déplacez les tenseurs sur le GPU
#     outputs = bert_model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).squeeze(0)  # Retourne le tenseur de caractéristiques


# batch_df4 = pd.DataFrame() 
# for batch_df in split_dataframe_into_batches(batch_df3, 10):  # Adjust batch size if necessary
#     batch_df['material_damage'] = batch_df['material_damage'].apply(text_to_features)
#     batch_df['circonstance_detail'] = batch_df['circonstance_detail'].apply(text_to_features)
#     batch_df['disaster_place'] = batch_df['disaster_place'].apply(text_to_features)
#     batch_df4 = pd.concat([batch_df4, batch_df], ignore_index=True)
#     del batch_df 
#     gc.collect()  
# print(f"Memory usage after: {get_memory_usage() / (1024 * 1024):.2f} MB") 


# # batch_df4 = pd.DataFrame()
# # for batch_df in split_dataframe_into_batches(batch_df3, 10):  # Ajustez la taille du lot si nécessaire
# #     for column in ['material_damage', 'circonstance_detail', 'disaster_place']:
# #         batch_df.loc[:, column] = batch_df[column].apply(text_to_features)
# #     batch_df4 = pd.concat([batch_df4, batch_df], ignore_index=True)
# #     del batch_df 
# #     gc.collect()

### BON sur GPU mais avec le modele de MiniLM ( Embbeding de dimension 386) plus de caracteristique

In [ ]:
%%cudf.pandas.profile
# import rmm
# rmm.reinitialize(pool_allocator=True, managed_memory=False)

print(f"Memory usage before: {get_memory_usage() / (1024 * 1024):.2f} MB")
import logging
from transformers import logging as transformers_logging
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import gc

# Configurer le niveau de logging pour ignorer les avertissements
transformers_logging.set_verbosity_error()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Chargement du modèle MiniLM
model_name = "microsoft/MiniLM-L12-H384-uncased"  # MiniLM
tokenizer = AutoTokenizer.from_pretrained(model_name)
miniLM_model = AutoModel.from_pretrained(model_name).to(device)

def text_to_features(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Déplacez les tenseurs sur le GPU
    outputs = miniLM_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze(0)  # Retourne le tenseur de caractéristiques


batch_df4 = pd.DataFrame() 
for batch_df in split_dataframe_into_batches(batch_df3, 10):  # Ajustez la taille du lot si nécessaire
    batch_df['material_damage'] = batch_df['material_damage'].apply(text_to_features)
    batch_df['circonstance_detail'] = batch_df['circonstance_detail'].apply(text_to_features)
    batch_df['disaster_place'] = batch_df['disaster_place'].apply(text_to_features)
    batch_df4 = pd.concat([batch_df4, batch_df], ignore_index=True)
    del batch_df 
    gc.collect()  

print(f"Memory usage after: {get_memory_usage() / (1024 * 1024):.2f} MB") 

In [ ]:
# Supposons que vous avez déjà extrait vos caractéristiques d'images et de textes
# %%cudf.pandas.profile
X_images_np_1 = np.array([feat.detach().numpy() for feat in batch_df4['feature_images_1']])  # Convertir en tableau NumPy
X_images_np_2 = np.array([feat.detach().numpy() for feat in batch_df4['feature_images_2']])
X_images_np_3 = np.array([feat.detach().numpy() for feat in batch_df4['feature_images_3']])
X_images_np_4 = np.array([feat.detach().numpy() for feat in batch_df4['feature_images_4']])
X_texts_np_1 = np.array([feat.detach().cpu().numpy() for feat in batch_df4['material_damage']])  # Convertir en tableau NumPy
X_texts_np_2 = np.array([feat.detach().cpu().numpy() for feat in batch_df4['circonstance_detail']]) 
X_texts_np_3 = np.array([feat.detach().cpu().numpy() for feat in batch_df4['disaster_place']]) 
X_marq_np = batch_df4['MARQVEHI'].to_numpy()
X_typevehi_np = batch_df4['TYPEVEHI'].to_numpy()
X_puivehi_np = batch_df4['PUISVEHI'].to_numpy()
X_nbreplace_np = batch_df4['NOMBPLAC'].to_numpy()
X_codegara_np = batch_df4['CODEGARA'].to_numpy()
X_codecat_np = batch_df4['CODECATE'].to_numpy()
# X_codebran_np = batch_df4['CODEBRAN'].to_numpy()
X_capri1_np = batch_df4['valeur_neuve'].to_numpy()
X_capri2_np = batch_df4['valeur_venale'].to_numpy() 
# X_texts_np_1 = np.array([feat for feat in batch_df4['material_damage']])
# Redimensionner les tableaux si nécessaire
X_images_np_1 = X_images_np_1.reshape(len(X_images_np_1), -1) # Redimensionner en 2D si c'est actuellement 1D
X_images_np_2 = X_images_np_2.reshape(len(X_images_np_2), -1)
X_images_np_3 = X_images_np_3.reshape(len(X_images_np_3), -1)
X_images_np_4 = X_images_np_4.reshape(len(X_images_np_4), -1)
X_texts_np_1 = X_texts_np_1.reshape(len(X_texts_np_1), -1) # Redimensionner en 2D si c'est actuellement 1D
X_texts_np_2 = X_texts_np_2.reshape(len(X_texts_np_2), -1)
X_texts_np_3 = X_texts_np_3.reshape(len(X_texts_np_3), -1) 
X_marq_np = X_marq_np.reshape(len(X_marq_np), -1)  # Redimensionner en 2D si c'est actuellement 1D
X_typevehi_np = X_typevehi_np.reshape(len(X_typevehi_np), -1)
X_puivehi_np = X_puivehi_np.reshape(len(X_puivehi_np), -1)  # Redimensionner en 2D si c'est actuellement 1D
X_nbreplace_np = X_nbreplace_np.reshape(len(X_nbreplace_np), -1)
X_codegara_np = X_codegara_np.reshape(len(X_codegara_np), -1)  # Redimensionner en 2D si c'est actuellement 1D
X_codecat_np = X_codecat_np.reshape(len(X_codecat_np), -1)
# X_codebran_np = X_codebran_np.reshape(len(X_codebran_np), -1)
X_capri1_np = X_capri1_np.reshape(len(X_capri1_np), -1)
X_capri2_np = X_capri2_np.reshape(len(X_capri2_np), -1)

# Concaténer ou combiner vos caractéristiques
X_combined = np.concatenate((X_texts_np_1, X_texts_np_2, X_texts_np_3, X_images_np_1, X_images_np_2, X_images_np_3, X_images_np_4, X_marq_np, X_typevehi_np, X_puivehi_np, X_nbreplace_np, X_codegara_np, X_codecat_np, X_capri1_np, X_capri2_np), axis=1) #X_capri2_np,  X_texts_np_3, X_images_np_4, X_texts_np_2, ,X_images_np_3 

# Labels ou cibles
y = batch_df4['cluster'].values

In [ ]:
columns_names = [f'feature_{i+1}' for i in range(X_combined.shape[1])]
X1_combined = pd.DataFrame(X_combined, columns = columns_names)

In [ ]:
X1_combined.shape

In [ ]:
df3 = pd.concat([X1_combined, batch_df4[['id', 'CODEINTE_NUMEPOLI_CODERISQ', 'COUTS_SINISTRES', 'cluster']]], axis=1)
df3.to_csv('/root/Gohou/Data/Tabes/New_trans/df21.csv') 

### ACP

In [20]:
n_files = 34 # Remplace ceci par le nombre exact de fichiers CSV que tu as

# Crée une liste des chemins des fichiers CSV
file_paths = [f'/root/Gohou/Data/Tabes/New_trans/df{i}.csv' for i in range(n_files)]

# Lire les fichiers CSV et stocker les DataFrames dans une liste
list_df = [pd.read_csv(file_path) for file_path in file_paths] 
print(len(list_df))
df_final = pd.concat(list_df, ignore_index=True) 

34


In [22]:
df = df_final.copy()
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [24]:
%%cudf.pandas.profile
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['Unnamed: 0', 'id', 'CODEINTE_NUMEPOLI_CODERISQ', 'COUTS_SINISTRES', 'cluster'], axis =1)  # Variables indépendantes
# Étape 1 : Normalisation des données
scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X)  # Normaliser tes données (X est ton jeu de données)

# Étape 2 : Application de l'ACP
pca = PCA(n_components = 0.8)  # Conserver 95% de la variance expliquée
X_pca = pca.fit_transform(X_scaled) 

# Afficher la variance expliquée par chaque composante principale
print("Variance expliquée par chaque composante principale : ", sum(pca.explained_variance_ratio_))
print("Nombre de composantes principales sélectionnées : ", pca.n_components_)   

Variance expliquée par chaque composante principale :  0.8001663613495702
Nombre de composantes principales sélectionnées :  1084


                                                                                                              
                                         Total time elapsed: 150.377 seconds                                  
                                      12837 GPU function calls in 3.963 seconds                               
                                        5 CPU function calls in 0.059 seconds                                 
                                                                                                              
                                                        Stats                                                 
                                                                                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                 ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame.drop           │ 1          │ 0.972       │ 0.972       │ 0          │ 0.000       │ 0.000       │
│ DataFrame                │ 2          │ 0.000       │ 0.000       │ 1          │ 0.001       │ 0.001       │
│ Series.apply             │ 0          │ 0.000       │ 0.000       │ 2          │ 0.036       │ 0.018       │
│ Series.any               │ 2          │ 0.040       │ 0.020       │ 0          │ 0.000       │ 0.000       │
│ Series.__len__           │ 0          │ 0.000       │ 0.000       │ 2          │ 0.023       │ 0.011       │
│ is_bool_dtype            │ 6416       │ 1.543       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ is_extension_array_dtype │ 6416       │ 1.407       │ 0.000       │ 0          │ 0.000       │ 0.000       │
└──────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- DataFrame
- Series.apply
- Series.__len__

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=653306;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

# X = df_final.drop(columns=['cluster','Unnamed: 0'])  # Variables indépendantes
# # Étape 1 : Normalisation des données
# scaler = StandardScaler() 
# X_scaled = scaler.fit_transform(X)  # Normaliser tes données (X est ton jeu de données)

# # Étape 2 : Application de l'ACP
# pca = PCA(n_components = 0.8)  # Conserver 95% de la variance expliquée
# X_pca = pca.fit_transform(X_scaled) 

# # Afficher la variance expliquée par chaque composante principale
# print("Variance expliquée par chaque composante principale : ", sum(pca.explained_variance_ratio_))
# print("Nombre de composantes principales sélectionnées : ", pca.n_components_)  

In [25]:
import pickle

# Enregistrer le modèle
with open('/root/Gohou/Data/Tabes/New_trans/ACP_SUP.pkl', 'wb') as acp:
    pickle.dump(pca, acp)  

with open('/root/Gohou/Data/Tabes/New_trans/scaler_SUP.pkl', 'wb') as scale:
    pickle.dump(scaler, scale)  

In [26]:
X_pca_df = pd.DataFrame(X_pca, columns=[f"PC{i+1}" for i in range(X_pca.shape[1])])
df1 = pd.concat([X_pca_df, df_final[['id', 'CODEINTE_NUMEPOLI_CODERISQ', 'COUTS_SINISTRES', 'cluster']]], axis=1) 
df1.shape

(6497, 1085)

In [42]:
# df1.to_csv('/root/Gohou/Data/Tabes/New_trans/Final.csv')

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# # Supposez que X et y sont vos données d'entrée et les labels
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Appliquer l'undersampling
# undersampler = RandomUnderSampler(random_state=42)
# X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# # Entraînez votre modèle avec les données undersamplées
# model = RandomForestClassifier(random_state=42)
# model.fit(X_resampled, y_resampled)

# # Faites des prédictions et évaluez le modèle
# y_pred = model.predict(X_test)
# print(classification_report(y_test, y_pred))

In [36]:
df = df1.copy()
from imblearn.over_sampling import SMOTE
from collections import Counter
X = df.drop(columns='cluster')  # Variables indépendantes
y = df['cluster'] 

# # Séparation des données en ensembles d'entraînement et de test
X_train1, X_test1, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)
X_train, X_test = X_train1.drop(columns=['id', 'CODEINTE_NUMEPOLI_CODERISQ', 'COUTS_SINISTRES']),  X_test1.drop(columns=['id', 'CODEINTE_NUMEPOLI_CODERISQ', 'COUTS_SINISTRES'])

# Application de SMOTE pour suréchantillonner la classe minoritaire
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Vérifier la distribution des classes après suréchantillonnage
print("Distribution des classes avant suréchantillonnage :", Counter(y_train))
print("Distribution des classes après suréchantillonnage :", Counter(y_train_resampled)) 

Distribution des classes avant suréchantillonnage : Counter({0: 2556, 1: 1327, 2: 626, 3: 363})
Distribution des classes après suréchantillonnage : Counter({1: 2556, 3: 2556, 0: 2556, 2: 2556})


In [ ]:
# !pip install seaborn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# # Séparation des données en ensembles d'entraînement et de test
X_train1, X_test1, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42 ,stratify=y)
X_train, X_test = X_train1.drop(columns=['id', 'CODEINTE_NUMEPOLI_CODERISQ', 'COUTS_SINISTRES']),  X_test1.drop(columns=['id', 'CODEINTE_NUMEPOLI_CODERISQ', 'COUTS_SINISTRES'])


# Appliquer l'undersampling
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Entraînez votre modèle avec les données undersamplées
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Faites des prédictions et évaluez le modèle
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred)) 

cm = confusion_matrix(y_test, y_pred)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100  # Convertir en pourcentages
# Affichage de la matrice de confusion avec les pourcentages
plt.figure(figsize=(8, 6))
sns.heatmap(cm_percent, annot=True, fmt=".2f", cmap="Blues", cbar=False)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (in %)')
plt.show()

In [ ]:
y_pred_series = pd.Series(y_pred, index=X_test1.index, name='y_pred')
y_test_series = pd.Series(y_test, index=X_test1.index, name='y_test')
a = pd.concat([X_test1[['id','CODEINTE_NUMEPOLI_CODERISQ','COUTS_SINISTRES']], y_test_series, y_pred_series], axis=1)
a.head() 

In [37]:
import time
import xgboost as xgb
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, KFold,cross_val_predict, train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression,LogisticRegression,ElasticNet, Ridge, Lasso
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [38]:
a = int(np.sqrt(len(X_train_resampled.columns))) 
dico = {
    'logistic': LogisticRegression(),
    'elastic': LogisticRegression(penalty="elasticnet", solver="saga"),  #met beaucoup de temps 
    'ridge': LogisticRegression(penalty="l2"),
    'lasso': LogisticRegression(penalty="l1"),
    'Arbre': DecisionTreeClassifier(),
    'Random': RandomForestClassifier(), 
    'xgb': xgb.XGBClassifier(),
    'knn': KNeighborsClassifier(),
    'Gradient_boosting': GradientBoostingClassifier(),
    'adaboost_classifier': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(DecisionTreeClassifier(max_depth=7)), 
    'SVM': SVC(),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis(),
    'lightgbm': LGBMClassifier(),  # Ajout de LightGBM
    'catboost': CatBoostClassifier(verbose=0)  # Ajout de CatBoost
}

param = [ 
    {"C": [0.01], "max_iter": [10000]},
    {"C": [0.01], "l1_ratio": [0.1, 0.5, 0.7], "max_iter": [10000]},
    {"C": [0.01], "solver": ["sag", "lbfgs", "liblinear"], "max_iter": [10000]}, # C grand surapprentissage 
    {"C": [0.01], "solver": ["liblinear"], "max_iter": [10000]},
    {"max_depth": [5, 10], "min_samples_split": [10, 15], "min_samples_leaf": [5, 7]},
    {"n_estimators": [200,300], "min_samples_leaf": [3, 5], "max_features": [0.7, 0.8], "max_depth": [8, 10]},
    {'colsample_bytree': [0.5, 0.7], "n_estimators": [50, 100], "max_depth": [2, 3], "objective": ["multi:softmax"], "num_class": [4], "subsample": [0.5, 0.6]}, 
    {'n_neighbors': [10, 15], "algorithm": ["auto","ball_tree"]}, # k petit surapprentissage 
    {"n_estimators": [800, 1000], "max_depth": [3], "learning_rate": [0.01, 0.05], "min_samples_split": [10, 15], "min_samples_leaf": [5, 7], "subsample": [0.5, 0.6]},
    {"n_estimators": [800, 1000], "learning_rate": [0.01, 0.05], "algorithm": ["SAMME", "SAMME.R"]},
    {"n_estimators": [50], "max_features": [0.5, 0.6], "max_samples":[0.5, 0.6]},
    {"kernel": ['linear', 'rbf'], "C": [0.1], "gamma": ["scale", "auto"]}, #C grand => surapprentisage
    {"solver": ["eigen", "lsqr"], "shrinkage": ["auto"]}, 
    {"reg_param": [0.7, 0.9], "tol": [1e-2, 0.1]},
    {"n_estimators": [1000, 800], "learning_rate": [0.01, 0.05], "max_depth": [2, 3], "num_leaves": [20, 30, 40], "colsample_bytree": [0.5, 0.6], "force_row_wise": [True], 'verbose': [-1]},
    {"iterations": [1000, 800], "depth": [3], "learning_rate": [0.01, 0.05], "l2_leaf_reg": [7, 10]}
] 

In [39]:
def train_model(nom_model, dico, param, X_train_resampled, y_train_resampled, kf, out):
    model = dico[nom_model]
    model_cv = GridSearchCV(model, param_grid=param, cv=kf, scoring="f1_weighted", error_score="raise")  # Utiliser "f1" au lieu de "f1_score"
    temps_debut = time.time() 
    model_cv.fit(X_train_resampled, y_train_resampled) 
    temps_fin = time.time()
    # temps_execution = temps_fin - temps_debut
    out[nom_model]={"best_model":model_cv.best_estimator_,"meilleurs_parametres":model_cv.best_params_, "f1_score": model_cv.best_score_,"temps":[time.ctime(temps_debut), time.ctime(temps_fin)]}
    print(f'-----------------  Entrainement de {nom_model} est terminé  -----------------') 

In [40]:
%%cudf.pandas.profile
import threading
kf = KFold(n_splits=5, shuffle=True, random_state=42)
out = {}
threads = []

# Launch a thread for each model in the dictionary
for i, nom_model in enumerate(dico):
    param_v = param[i]
    thread = threading.Thread(target=train_model, args=(nom_model, dico, param_v, X_train_resampled, y_train_resampled, kf, out))
    threads.append(thread)
    thread.start() 

                                                                                              
                                  Total time elapsed: 0.154 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Exception in thread Thread-20 (train_model):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_3835/99327755.py", line 5, in train_model
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py", line 970, in fit
    self._run_search(evaluate_candidates)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py", line 1527, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py

-----------------  Entrainement de lasso est terminé  -----------------
-----------------  Entrainement de logistic est terminé  -----------------
-----------------  Entrainement de LDA est terminé  -----------------
-----------------  Entrainement de ridge est terminé  -----------------
-----------------  Entrainement de elastic est terminé  -----------------
-----------------  Entrainement de QDA est terminé  -----------------
-----------------  Entrainement de knn est terminé  -----------------
-----------------  Entrainement de SVM est terminé  -----------------
-----------------  Entrainement de Arbre est terminé  -----------------
-----------------  Entrainement de xgb est terminé  -----------------
-----------------  Entrainement de Bagging est terminé  -----------------


In [78]:
out_1  = pd.DataFrame.from_dict(out, orient='index')
out_1

,best_model,meilleurs_parametres,f1_score,temps
lasso,"LogisticRegression(C=0.01, max_iter=10000, pen...","{'C': 0.01, 'max_iter': 10000, 'solver': 'libl...",0.586748,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 14:06:35..."
logistic,"LogisticRegression(C=0.01, max_iter=10000)","{'C': 0.01, 'max_iter': 10000}",0.711966,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 14:08:28..."
LDA,"LinearDiscriminantAnalysis(shrinkage='auto', s...","{'shrinkage': 'auto', 'solver': 'eigen'}",0.677255,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 14:28:13..."
ridge,"LogisticRegression(C=0.01, max_iter=10000)","{'C': 0.01, 'max_iter': 10000, 'solver': 'lbfgs'}",0.711966,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 14:49:47..."
elastic,"LogisticRegression(C=0.01, l1_ratio=0.1, max_i...","{'C': 0.01, 'l1_ratio': 0.1, 'max_iter': 10000}",0.703424,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 14:58:15..."
QDA,"QuadraticDiscriminantAnalysis(reg_param=0.7, t...","{'reg_param': 0.7, 'tol': 0.01}",0.923993,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 15:01:43..."
knn,KNeighborsClassifier(n_neighbors=10),"{'algorithm': 'auto', 'n_neighbors': 10}",0.503680,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 15:07:44..."
SVM,"SVC(C=0.1, kernel='linear')","{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}",0.788589,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 15:19:51..."
Arbre,"DecisionTreeClassifier(max_depth=10, min_sampl...","{'max_depth': 10, 'min_samples_leaf': 5, 'min_...",0.514351,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 15:23:00..."
xgb,"XGBClassifier(base_score=None, booster=None, c...","{'colsample_bytree': 0.5, 'max_depth': 3, 'n_e...",0.811508,"[Thu Oct 24 13:34:45 2024, Thu Oct 24 16:07:49..."


In [ ]:
# Pour s'assurer que tous les threads individuelles soient terminés avant que le thread principal prenne le relai. 
# On en pas besoin si on sait à quel moment tous nos threads vont tous se stopper 

for thread in threads:
    thread.join() 
print(out) 

In [77]:
pred = out_1.loc["Bagging","best_model"].predict(X_train_resampled)
f1_test = f1_score(y_train_resampled, pred, average='weighted')
# X_train_resampled, y_train_resampled
# f1_test = f1_score(y_train_resampled, pred, average='weighted')
# f1_test = accuracy_score(y_test, pred)
print(f1_test)  

0.9389331960668086


In [75]:
 out_1.loc["Bagging","meilleurs_parametres"]

{'max_features': 0.5, 'max_samples': 0.6, 'n_estimators': 80}